# Siren Exploration

This is a colab to explore properties of the Siren MLP, proposed in our work [Implicit Neural Activations with Periodic Activation Functions](https://vsitzmann.github.io/siren).


We will first implement a streamlined version of Siren for fast experimentation. This lacks the code to easily do baseline comparisons - please refer to the main code for that - but will greatly simplify the code!

**Make sure that you have enabled the GPU under Edit -> Notebook Settings!**

We will then reproduce the following results from the paper: 
* [Fitting an image](#section_1)
* [Fitting an audio signal](#section_2)
* [Solving Poisson's equation](#section_3)
* [Initialization scheme & distribution of activations](#activations)
* [Distribution of activations is shift-invariant](#shift_invariance)

We will also explore Siren's [behavior outside of the training range](#out_of_range).

Let's go! First, some imports, and a function to quickly generate coordinate grids.

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os

from PIL import Image
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
import numpy as np
import skimage
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from math import log10, sqrt

import time

def get_mgrid(sidelen, dim=2):  #function to generate the coordinate grid
    '''Generates a flattened grid of (x,y,...) coordinates in a range of -1 to 1.
    sidelen: int
    dim: int'''

    #torch.linspace(-1, 1, steps=sidelen):
    #Creates a one-dimensional tensor of size steps whose values are evenly
    #spaced from start (=-1) to end (=1), inclusive.

    #dim * [..]
    #replicate the content of [] dim times, the result will be a list of dim
    #elements (in this case tensors)

    #tuple([..])
    #transforms the list in input in a tuple

    tensors = tuple(dim * [torch.linspace(-1, 1, steps=sidelen)])

    #torch.meshgrid(*tensors):
    #Take N tensors, each of which can be either scalar or 1-dimensional vector,
    #and create N N-dimensional grids, where the i-th grid is defined by
    #expanding the i-th input over dimensions defined by other inputs.

    #torch.stack(...,dim=-1):
    #Concatenates a sequence of tensors along a new dimension
    #dim = -1 -> calculate the dimensions automatically

    mgrid = torch.stack(torch.meshgrid(*tensors), dim=-1)

    #mgrid.reshape(-1, dim):
    #Returns a tensor with the same data and number of elements as input,
    #but with the specified shape.
    #dim = -1 -> calculate the dimensions automatically

    #from a list of lists (3 dimensions) to a single list (2 dimensions)

    mgrid = mgrid.reshape(-1, dim)
    return mgrid

def psnr(input, target):
    MSE = ((input - target)**2).mean() 
    PSNR_reconstructed_image = 20*log10(1/sqrt(MSE.item()))
    return PSNR_reconstructed_image

In [2]:
#my_test
print([torch.linspace(-1, 1, steps=5)])
print(2 * [torch.linspace(-1, 1, steps=5)])
tensors = tuple(2 * [torch.linspace(-1, 1, steps=5)])
print(tensors)
torch.meshgrid(*tensors)
mgrid = torch.stack(torch.meshgrid(*tensors), dim=-1)
print(mgrid)
print(mgrid.reshape(-1, 2))

print(get_mgrid(256, 2))
print(get_mgrid(256, 2).shape)

[tensor([-1.0000, -0.5000,  0.0000,  0.5000,  1.0000])]
[tensor([-1.0000, -0.5000,  0.0000,  0.5000,  1.0000]), tensor([-1.0000, -0.5000,  0.0000,  0.5000,  1.0000])]
(tensor([-1.0000, -0.5000,  0.0000,  0.5000,  1.0000]), tensor([-1.0000, -0.5000,  0.0000,  0.5000,  1.0000]))
tensor([[[-1.0000, -1.0000],
         [-1.0000, -0.5000],
         [-1.0000,  0.0000],
         [-1.0000,  0.5000],
         [-1.0000,  1.0000]],

        [[-0.5000, -1.0000],
         [-0.5000, -0.5000],
         [-0.5000,  0.0000],
         [-0.5000,  0.5000],
         [-0.5000,  1.0000]],

        [[ 0.0000, -1.0000],
         [ 0.0000, -0.5000],
         [ 0.0000,  0.0000],
         [ 0.0000,  0.5000],
         [ 0.0000,  1.0000]],

        [[ 0.5000, -1.0000],
         [ 0.5000, -0.5000],
         [ 0.5000,  0.0000],
         [ 0.5000,  0.5000],
         [ 0.5000,  1.0000]],

        [[ 1.0000, -1.0000],
         [ 1.0000, -0.5000],
         [ 1.0000,  0.0000],
         [ 1.0000,  0.5000],
         [ 1.0000,

Now, we code up the sine layer, which will be the basic building block of SIREN. This is a much more concise implementation than the one in the main code, as here, we aren't concerned with the baseline comparisons.

In [3]:
class SineLayer(nn.Module):
    # See paper sec. 3.2, final paragraph, and supplement Sec. 1.5 for discussion of omega_0.
    
    # If is_first=True, omega_0 is a frequency factor which simply multiplies the activations before the 
    # nonlinearity. Different signals may require different omega_0 in the first layer - this is a 
    # hyperparameter.
    
    # If is_first=False, then the weights will be divided by omega_0 so as to keep the magnitude of 
    # activations constant, but boost gradients to the weight matrix (see supplement Sec. 1.5)
    
    def __init__(self, in_features, out_features, bias=True,
                 is_first=False, omega_0=30):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        
        self.init_weights()
    
    def init_weights(self):
        with torch.no_grad(): #Context-manager that disables gradient calculation
            if self.is_first:
                #Fills the input Tensor with values drawn from the uniform distribution

                #Initialize weights
                self.linear.weight.uniform_(-1 / self.in_features, 
                                             1 / self.in_features)
            else:
                #Fills the input Tensor with values drawn from the uniform distribution

                #Initialize weights
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0, 
                                             np.sqrt(6 / self.in_features) / self.omega_0)
        
    def forward(self, input): #forward pass
        return torch.sin(self.omega_0 * self.linear(input))
    
    def forward_with_intermediate(self, input): #forward pass with intermediate
        # For visualization of activation distributions
        intermediate = self.omega_0 * self.linear(input)
        return torch.sin(intermediate), intermediate
    
    
class Siren(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features, outermost_linear=False, 
                 first_omega_0=30, hidden_omega_0=30.):
        super().__init__()
        
        self.net = []   #net

        #append the first sine layer
        self.net.append(SineLayer(in_features, hidden_features, 
                                  is_first=True, omega_0=first_omega_0))

        #append all the other sine layers
        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features, 
                                      is_first=False, omega_0=hidden_omega_0))

        #if the outermost_linear bool is set then add a final linear layer
        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)
            
            with torch.no_grad(): #Context-manager that disables gradient calculation
                final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0, 
                                              np.sqrt(6 / hidden_features) / hidden_omega_0)
                
            self.net.append(final_linear)
        else:
            self.net.append(SineLayer(hidden_features, out_features, 
                                      is_first=False, omega_0=hidden_omega_0))
        
        #nn.Sequential(*self.net):
        #A sequential container. Modules will be added to it in the order they
        #are passed in the constructor.

        self.net = nn.Sequential(*self.net)
    
    def forward(self, coords):  #forward pass
        coords = coords.clone().detach().requires_grad_(True) # allows to take derivative w.r.t. input
        output = self.net(coords) #apply coords on net
        return output, coords        

    def forward_with_activations(self, coords, retain_grad=False):  #forward pass with activations
        '''Returns not only model output, but also intermediate activations.
        Only used for visualizing activations later!'''
        activations = OrderedDict()

        activation_count = 0
        x = coords.clone().detach().requires_grad_(True)  # allows to take derivative w.r.t. input
        activations['input'] = x
        for i, layer in enumerate(self.net):  #for all layers of the net
            if isinstance(layer, SineLayer):  #if it is a sine layer
                x, intermed = layer.forward_with_intermediate(x)
                
                #retain_grad():
                #Enables .grad attribute for non-leaf Tensors.

                if retain_grad:
                    x.retain_grad()
                    intermed.retain_grad()
                    
                activations['_'.join((str(layer.__class__), "%d" % activation_count))] = intermed
                activation_count += 1
            else: 
                x = layer(x)
                
                if retain_grad:
                    x.retain_grad()
                    
            activations['_'.join((str(layer.__class__), "%d" % activation_count))] = x
            activation_count += 1

        return activations

And finally, differential operators that allow us to leverage autograd to compute gradients, the laplacian, etc.

In [4]:
def laplace(y, x):
    grad = gradient(y, x)
    return divergence(grad, x)


def divergence(y, x):
    div = 0.
    for i in range(y.shape[-1]):
        #torch.autograd.grad(...)
        #Computes and returns the sum of gradients of outputs w.r.t. the inputs.
        div += torch.autograd.grad(y[..., i], x, torch.ones_like(y[..., i]), create_graph=True)[0][..., i:i+1]
    return div


def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        #torch.ones_like(input)
        #Returns a tensor filled with the scalar value 1, with the same size as input.
        grad_outputs = torch.ones_like(y)

    #torch.autograd.grad(...)
    #Computes and returns the sum of gradients of outputs w.r.t. the inputs.
    grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os

from PIL import Image
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
import numpy as np
import skimage
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from math import log10, sqrt

import time

def get_mgrid(sidelen, dim=2):  #function to generate the coordinate grid
    '''Generates a flattened grid of (x,y,...) coordinates in a range of -1 to 1.
    sidelen: int
    dim: int'''

    #torch.linspace(-1, 1, steps=sidelen):
    #Creates a one-dimensional tensor of size steps whose values are evenly
    #spaced from start (=-1) to end (=1), inclusive.

    #dim * [..]
    #replicate the content of [] dim times, the result will be a list of dim
    #elements (in this case tensors)

    #tuple([..])
    #transforms the list in input in a tuple

    tensors = tuple(dim * [torch.linspace(-1, 1, steps=sidelen)])

    #torch.meshgrid(*tensors):
    #Take N tensors, each of which can be either scalar or 1-dimensional vector,
    #and create N N-dimensional grids, where the i-th grid is defined by
    #expanding the i-th input over dimensions defined by other inputs.

    #torch.stack(...,dim=-1):
    #Concatenates a sequence of tensors along a new dimension
    #dim = -1 -> calculate the dimensions automatically

    mgrid = torch.stack(torch.meshgrid(*tensors), dim=-1)

    #mgrid.reshape(-1, dim):
    #Returns a tensor with the same data and number of elements as input,
    #but with the specified shape.
    #dim = -1 -> calculate the dimensions automatically

    #from a list of lists (3 dimensions) to a single list (2 dimensions)

    mgrid = mgrid.reshape(-1, dim)
    return mgrid

In [6]:
def get_cameraman_tensor(sidelength):
    #skimage.data.camera():
    #Gray-level “camera” image

    img = Image.fromarray(skimage.data.camera())        
    transform = Compose([
        Resize(sidelength),
        ToTensor(),
        Normalize(torch.Tensor([0.5]), torch.Tensor([0.5])) #normalize
    ])
    img = transform(img)  #image tensor of dimension (1, sidelength, sidelength)
    return img

In [7]:
cameraman = get_cameraman_tensor(256)
print(cameraman)
print(cameraman.shape)

print()
print(cameraman.permute(1, 2, 0))
print(cameraman.permute(1, 2, 0).shape)

print()
print(cameraman.permute(1, 2, 0).view(-1, 1))
print(cameraman.permute(1, 2, 0).view(-1, 1).shape)

tensor([[[ 0.2314,  0.2392,  0.2314,  ...,  0.1843,  0.1843,  0.1922],
         [ 0.2314,  0.2235,  0.2314,  ...,  0.2000,  0.2000,  0.1922],
         [ 0.2314,  0.2314,  0.2235,  ...,  0.1922,  0.2000,  0.2000],
         ...,
         [-0.0431, -0.0118, -0.0667,  ...,  0.0510,  0.0039, -0.0980],
         [-0.0431, -0.0039,  0.0431,  ...,  0.0431, -0.0118, -0.1137],
         [-0.0353, -0.0118,  0.1137,  ...,  0.0353, -0.0196, -0.1137]]])
torch.Size([1, 256, 256])

tensor([[[ 0.2314],
         [ 0.2392],
         [ 0.2314],
         ...,
         [ 0.1843],
         [ 0.1843],
         [ 0.1922]],

        [[ 0.2314],
         [ 0.2235],
         [ 0.2314],
         ...,
         [ 0.2000],
         [ 0.2000],
         [ 0.1922]],

        [[ 0.2314],
         [ 0.2314],
         [ 0.2235],
         ...,
         [ 0.1922],
         [ 0.2000],
         [ 0.2000]],

        ...,

        [[-0.0431],
         [-0.0118],
         [-0.0667],
         ...,
         [ 0.0510],
         [ 0.00

<a id='section_1'></a>
## *1.* Fitting an image

First, let's simply fit that image!

We seek to parameterize a greyscale image $f(x)$ with pixel coordinates $x$ with a SIREN $\Phi(x)$.

That is we seek the function $\Phi$ such that:
$\mathcal{L}=\int_{\Omega} \lVert \Phi(\mathbf{x}) - f(\mathbf{x}) \rVert\mathrm{d}\mathbf{x}$
 is minimized, in which $\Omega$ is the domain of the image. 
 
We write a little datast that does nothing except calculating per-pixel coordinates:

In [8]:
class ImageFitting(Dataset):
    def __init__(self, sidelength):
        super().__init__()

        #get the cameraman tensor with dimension sidelength
        #image tensor of dimension (1, sidelength, sidelength)
        img = get_cameraman_tensor(sidelength)

        #tensor.permute(*dims):
        #Returns a view of the original tensor with its dimensions permuted

        #I obtain an image tensor of dimension (sidelength, sidelength, 1)

        #tensor.view(*shape)
        #Returns a new tensor with the same data as the self tensor but of a
        #different shape
        #dim = -1 -> calculate the dimensions automatically

        #to obtain an image tensor of dimension (sidelength x sidelength, 1)
        
        self.pixels = img.permute(1, 2, 0).view(-1, 1)
        #I obtain an image tensor of dimension (sidelength x sidelength, 1)
        #this represents the y (output) value for each pixel

        #get mgrid of the same dimension of the image (sidelength)
        self.coords = get_mgrid(sidelength, 2)
        #I obtain a grid tensor of dimension (sidelength x sidelength, 2)
        #this represents the (x1, x2) (input) pair for each pixel

    def __len__(self):
        return 1

    def __getitem__(self, idx):    
        if idx > 0: raise IndexError
            
        return self.coords, self.pixels

Let's instantiate the dataset and our Siren. As pixel coordinates are 2D, the siren has 2 input features, and since the image is grayscale, it has one output channel.

In [9]:
cameraman = ImageFitting(256)

#instantiate data loader with a single batch (we have only one image)
dataloader = DataLoader(cameraman, batch_size=1, pin_memory=True, num_workers=0)

#instantiate siren net with number of input features = 2 ((x1,x2) pairs),
#number of output features = 1 (y value), hidden features, hidden layers
#and if to have the last layer as a linear layer or not
img_siren = Siren(in_features=2, out_features=1, hidden_features=256, 
                  hidden_layers=2, outermost_linear=True)

img_siren.cuda()

Siren(
  (net): Sequential(
    (0): SineLayer(
      (linear): Linear(in_features=2, out_features=256, bias=True)
    )
    (1): SineLayer(
      (linear): Linear(in_features=256, out_features=256, bias=True)
    )
    (2): SineLayer(
      (linear): Linear(in_features=256, out_features=256, bias=True)
    )
    (3): Linear(in_features=256, out_features=1, bias=True)
  )
)

We now fit Siren in a simple training loop. Within only hundreds of iterations, the image and its gradients are approximated well.

In [10]:
total_steps = 500 # Since the whole image is our dataset, this just means 500 gradient descent steps.
steps_til_summary = 10 #steps at which print image and summary

#define optimizer
optim = torch.optim.Adam(lr=1e-4, params=img_siren.parameters())

#model_input is the grid (of (x1,x2) pairs)
#ground_truth is the image (y values)
model_input, ground_truth = next(iter(dataloader))
model_input, ground_truth = model_input.cuda(), ground_truth.cuda()

for step in range(total_steps):

    #model_output is the output of the single forward pass, it is an output image
    #tensor of dimension (sidelength x sidelength, 1);
    #this represents the y^ (output) value for each pixel

    #coords is a copy of the model_input pairs
    model_output, coords = img_siren(model_input) #forward pass 

    #model_output and ground_truth (original image) have the same dimensions
    loss = ((model_output - ground_truth)**2).mean()  #calculate loss (MSE)
    the_psnr = psnr(ground_truth, model_output)
    
    if not step % steps_til_summary:
        print("Step %d, Total MSR loss %0.6f" % (step, loss))
        img_grad = gradient(model_output, coords) #calculate output image gradient
        img_laplacian = laplace(model_output, coords) #calculate output image laplacian

        fig, axes = plt.subplots(1,4, figsize=(18,6))
        axes[0].set_title("PSNR: %0.4f " % the_psnr)
        axes[0].imshow(model_output.cpu().view(256,256).detach().numpy())
        axes[1].set_title("Gradient")
        axes[1].imshow(img_grad.norm(dim=-1).cpu().view(256,256).detach().numpy())
        axes[2].set_title("Laplacian")
        axes[2].imshow(img_laplacian.cpu().view(256,256).detach().numpy())
        axes[3].set_title("Ground Truth")
        axes[3].imshow(ground_truth.cpu().view(256,256).detach().numpy())

        plt.show()

    optim.zero_grad() #Sets the gradients of all optimized torch.Tensor to zero
    loss.backward() ##apply gradients
    optim.step()  #make optimizer step

'\ntotal_steps = 500 # Since the whole image is our dataset, this just means 500 gradient descent steps.\nsteps_til_summary = 10 #steps at which print image and summary\n\n#define optimizer\noptim = torch.optim.Adam(lr=1e-4, params=img_siren.parameters())\n\n#model_input is the grid (of (x1,x2) pairs)\n#ground_truth is the image (y values)\nmodel_input, ground_truth = next(iter(dataloader))\nmodel_input, ground_truth = model_input.cuda(), ground_truth.cuda()\n\nfor step in range(total_steps):\n\n    #model_output is the output of the single forward pass, it is an output image\n    #tensor of dimension (sidelength x sidelength, 1);\n    #this represents the y^ (output) value for each pixel\n\n    #coords is a copy of the model_input pairs\n    model_output, coords = img_siren(model_input) #forward pass \n\n    #model_output and ground_truth (original image) have the same dimensions\n    loss = ((model_output - ground_truth)**2).mean()  #calculate loss (MSE)\n    the_psnr = psnr(ground_t

<a id='section_3'></a>
## *2.* Solving Poisson's equation

Now, let's make it a bit harder. Let's say we want to reconstruct an image but we only have access to its gradients!

That is, we now seek the function $\Phi$ such that:
$\mathcal{L}=\int_{\Omega} \lVert \nabla\Phi(\mathbf{x}) - \nabla f(\mathbf{x}) \rVert\mathrm{d}\mathbf{x}$
 is minimized, in which $\Omega$ is the domain of the image. 

In [11]:
import scipy.ndimage
    
class PoissonEqn(Dataset):
    def __init__(self, sidelength):
        super().__init__()

        #get the cameraman tensor with dimension sidelength
        #image tensor of dimension (1, sidelength, sidelength)
        img = get_cameraman_tensor(sidelength)
        
        # Compute gradient and laplacian  

        #scipy.ndimage.sobel(...)
        #Calculate a Sobel filter (axis is the axis of input along which to calculate)
        #returns a ndarray of dimension (1, sidelength, sidelength) in this case

        #torch.squeeze(0)
        #Remove single-dimensional entries from the shape of a (along the 0 axis)
        #returns a ndarray of dimension (sidelength, sidelength) in this case

        #ndarray [...,None]
        #changes the ndarray shape
        #returns a ndarray of dimension (sidelength, sidelength, 1) in this case

        grads_x = scipy.ndimage.sobel(img.numpy(), axis=1).squeeze(0)[..., None]
        grads_y = scipy.ndimage.sobel(img.numpy(), axis=2).squeeze(0)[..., None]

        #torch.from_numpy
        #Creates a Tensor from a numpy.ndarray

        grads_x, grads_y = torch.from_numpy(grads_x), torch.from_numpy(grads_y)
                
        #torch.stack(...,dim=-1):
        #Concatenates a sequence of tensors along a new dimension
        #dim = -1 -> calculate the dimensions automatically

        #tensor.view(*shape)
        #Returns a new tensor with the same data as the self tensor but of a
        #different shape
        #dim = -1 -> calculate the dimensions automatically

        #to obtain a grad tensor of dimension (sidelength x sidelength, 2)

        self.grads = torch.stack((grads_x, grads_y), dim=-1).view(-1, 2)

        #scipy.ndimage.laplace
        #N-D Laplace filter based on approximate second derivatives
        #returns a ndarray of dimension (1, sidelength, sidelength) in this case

        #torch.squeeze(0)
        #Remove single-dimensional entries from the shape of a (along the 0 axis)
        #returns a ndarray of dimension (sidelength, sidelength) in this case

        #ndarray [...,None]
        #changes the ndarray shape
        #returns a ndarray of dimension (sidelength, sidelength, 1) in this case

        self.laplace = scipy.ndimage.laplace(img.numpy()).squeeze(0)[..., None]

        #torch.from_numpy
        #Creates a Tensor from a numpy.ndarray

        self.laplace = torch.from_numpy(self.laplace)

        #tensor.permute(*dims):
        #Returns a view of the original tensor with its dimensions permuted

        #I obtain an image tensor of dimension (sidelength, sidelength, 1)

        #tensor.view(*shape)
        #Returns a new tensor with the same data as the self tensor but of a
        #different shape
        #dim = -1 -> calculate the dimensions automatically

        #to obtain an image tensor of dimension (sidelength x sidelength, 1)
        
        self.pixels = img.permute(1, 2, 0).view(-1, 1)
        #I obtain an image tensor of dimension (sidelength x sidelength, 1)
        #this represents the y (output) value for each pixel

        #get mgrid of the same dimension of the image (sidelength)
        self.coords = get_mgrid(sidelength, 2)
        #I obtain a grid tensor of dimension (sidelength x sidelength, 2)
        #this represents the (x1, x2) (input) pair for each pixel

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        return self.coords, {'pixels':self.pixels, 'grads':self.grads, 'laplace':self.laplace}

In [12]:
import scipy.ndimage

img = get_cameraman_tensor(256)
grads_x_1 = scipy.ndimage.sobel(img.numpy(), axis=1)

print(grads_x_1)
print()

grads_x_2 = scipy.ndimage.sobel(img.numpy(), axis=1).squeeze(0)

print(grads_x_2)
print()

grads_x = scipy.ndimage.sobel(img.numpy(), axis=1).squeeze(0)[..., None]
grads_y = scipy.ndimage.sobel(img.numpy(), axis=2).squeeze(0)[..., None]

print(grads_x)
print(grads_y)

grads_x, grads_y = torch.from_numpy(grads_x), torch.from_numpy(grads_y)

print(torch.stack((grads_x, grads_y), dim=-1))
grads = torch.stack((grads_x, grads_y), dim=-1).view(-1, 2)
print(grads)

[[[-0.06274509 -0.12549019 -0.03137255 ...  0.25098038  0.18823528
    0.06274509]
  [-0.03137255 -0.09411764 -0.12549019 ...  0.15686274  0.18823528
    0.15686274]
  [ 0.         -0.03137255 -0.09411764 ... -0.03137255  0.03137255
    0.09411764]
  ...
  [ 0.8784313   1.3490198   2.1333337  ...  1.3490202   0.40784335
   -0.18823528]
  [ 0.09411764  0.75294137  2.7294123  ... -0.65882397 -0.31372595
   -0.28235316]
  [ 0.06274509  0.25098038  1.0039217  ... -0.3764708  -0.09411764
   -0.03137255]]]

[[-0.06274509 -0.12549019 -0.03137255 ...  0.25098038  0.18823528
   0.06274509]
 [-0.03137255 -0.09411764 -0.12549019 ...  0.15686274  0.18823528
   0.15686274]
 [ 0.         -0.03137255 -0.09411764 ... -0.03137255  0.03137255
   0.09411764]
 ...
 [ 0.8784313   1.3490198   2.1333337  ...  1.3490202   0.40784335
  -0.18823528]
 [ 0.09411764  0.75294137  2.7294123  ... -0.65882397 -0.31372595
  -0.28235316]
 [ 0.06274509  0.25098038  1.0039217  ... -0.3764708  -0.09411764
  -0.03137255]]



#### Instantiate SIREN model

In [13]:
cameraman_poisson = PoissonEqn(256)

#instantiate data loader with a single batch (we have only one image)
dataloader = DataLoader(cameraman_poisson, batch_size=1, pin_memory=True, num_workers=0)

#instantiate siren net with number of input features = 2 ((x1,x2) pairs),
#number of output features = 1 (y value), hidden features, hidden layers
#and if to have the last layer as a linear layer or not
poisson_siren = Siren(in_features=2, out_features=1, hidden_features=256, 
                      hidden_layers=3, outermost_linear=True)
poisson_siren.cuda()

Siren(
  (net): Sequential(
    (0): SineLayer(
      (linear): Linear(in_features=2, out_features=256, bias=True)
    )
    (1): SineLayer(
      (linear): Linear(in_features=256, out_features=256, bias=True)
    )
    (2): SineLayer(
      (linear): Linear(in_features=256, out_features=256, bias=True)
    )
    (3): SineLayer(
      (linear): Linear(in_features=256, out_features=256, bias=True)
    )
    (4): Linear(in_features=256, out_features=1, bias=True)
  )
)

#### Define the loss function

In [14]:
def gradients_mse(model_output, coords, gt_gradients):
    # compute gradients on the model

    #calculate the gradient from the output image of the model
    gradients = gradient(model_output, coords)

    # compare them with the ground-truth gradients (mean squared error)
    gradients_loss = torch.mean((gradients - gt_gradients).pow(2).sum(-1))
    return gradients_loss

#### Train the model (from gradient)

In [15]:
total_steps = 500  # Since the whole image is our dataset, this just means 500 gradient descent steps.
steps_til_summary = 10  #steps at which print image and summary

#define optimizer
optim = torch.optim.Adam(lr=1e-4, params=poisson_siren.parameters())

#model_input is the grid (of (x1,x2) pairs)
#ground truth is the image (y values)
model_input, gt = next(iter(dataloader))

#ground truth is composed of 3 terms (pixels, grads, laplace), move them to cuda
gt = {key: value.cuda() for key, value in gt.items()}

model_input = model_input.cuda()

for step in range(total_steps):
    start_time = time.time()

    #model_output is the output of the single forward pass, it is an output image
    #tensor of dimension (sidelength x sidelength, 1);
    #this represents the y^ (output) value for each pixel

    #coords is a copy of the model_input pairs  
    model_output, coords = poisson_siren(model_input) #forward pass 

    #model_output and ground_truth (original image) have the same dimensions

    #calculate MSE
    train_loss = gradients_mse(model_output, coords, gt['grads'])
    the_psnr = psnr(gt['pixels'], model_output)

    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f, iteration time %0.6f" % (step, train_loss, time.time() - start_time))

        img_grad = gradient(model_output, coords) #calculate output image gradient
        img_laplacian = laplace(model_output, coords) #calculate output image laplacian

        fig, axes = plt.subplots(2,3, figsize=(18,10))
        axes[0, 0].set_title("PSNR: %0.4f " % the_psnr)
        axes[0, 0].imshow(model_output.cpu().view(256,256).detach().numpy())
        axes[0, 1].set_title("Our Gradient")
        axes[0, 1].imshow(img_grad.cpu().norm(dim=-1).view(256,256).detach().numpy())
        axes[0, 2].set_title("Our Laplacian")
        axes[0, 2].imshow(img_laplacian.cpu().view(256,256).detach().numpy())
        axes[1, 0].set_title("Ground Truth")
        axes[1, 0].imshow(gt['pixels'].cpu().view(256,256).detach().numpy())
        axes[1, 1].set_title("Ground Truth Gradient")
        axes[1, 1].imshow(gt['grads'].norm(dim=-1).cpu().view(256,256).detach().numpy())
        axes[1, 2].set_title("Ground Truth Laplacian")
        axes[1, 2].imshow(gt['laplace'].cpu().view(256,256).detach().numpy())
        plt.show()
        
    optim.zero_grad() #Sets the gradients of all optimized torch.Tensor to zero
    train_loss.backward() #apply gradients
    optim.step()  #make optimizer step

'\ntotal_steps = 500  # Since the whole image is our dataset, this just means 500 gradient descent steps.\nsteps_til_summary = 10  #steps at which print image and summary\n\n#define optimizer\noptim = torch.optim.Adam(lr=1e-4, params=poisson_siren.parameters())\n\n#model_input is the grid (of (x1,x2) pairs)\n#ground truth is the image (y values)\nmodel_input, gt = next(iter(dataloader))\n\n#ground truth is composed of 3 terms (pixels, grads, laplace), move them to cuda\ngt = {key: value.cuda() for key, value in gt.items()}\n\nmodel_input = model_input.cuda()\n\nfor step in range(total_steps):\n    start_time = time.time()\n\n    #model_output is the output of the single forward pass, it is an output image\n    #tensor of dimension (sidelength x sidelength, 1);\n    #this represents the y^ (output) value for each pixel\n\n    #coords is a copy of the model_input pairs  \n    model_output, coords = poisson_siren(model_input) #forward pass \n\n    #model_output and ground_truth (original 

#### Train the model (from laplacian)

> Blocco con rientro



In [16]:
poisson_siren = Siren(in_features=2, out_features=1, hidden_features=256, 
                      hidden_layers=3, outermost_linear=True)
poisson_siren.cuda()

def laplace_mse(model_output, coords, gt_laplace):
    # compute gradients on the model

    #calculate the gradient from the output image of the model
    laplacian = laplace(model_output, coords)

    # compare them with the ground-truth gradients (mean squared error)
    laplace_loss = torch.mean((laplacian.view(256,256) - gt_laplace.view(256,256)).pow(2).sum(-1))
    return laplace_loss

total_steps = 500  # Since the whole image is our dataset, this just means 500 gradient descent steps.
steps_til_summary = 10  #steps at which print image and summary

#define optimizer
optim = torch.optim.Adam(lr=1e-4, params=poisson_siren.parameters())

#model_input is the grid (of (x1,x2) pairs)
#ground truth is the image (y values)
model_input, gt = next(iter(dataloader))

#ground truth is composed of 3 terms (pixels, grads, laplace), move them to cuda
gt = {key: value.cuda() for key, value in gt.items()}

model_input = model_input.cuda()

for step in range(total_steps):
    start_time = time.time()

    #model_output is the output of the single forward pass, it is an output image
    #tensor of dimension (sidelength x sidelength, 1);
    #this represents the y^ (output) value for each pixel

    #coords is a copy of the model_input pairs  
    model_output, coords = poisson_siren(model_input) #forward pass 

    #model_output and ground_truth (original image) have the same dimensions

    #calculate MSE
    train_loss = laplace_mse(model_output, coords, gt['laplace'])
    the_psnr = psnr(gt['pixels'], model_output)

    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f, iteration time %0.6f" % (step, train_loss, time.time() - start_time))

        img_grad = gradient(model_output, coords) #calculate output image gradient
        img_laplacian = laplace(model_output, coords) #calculate output image laplacian

        fig, axes = plt.subplots(2,3, figsize=(18,10))
        axes[0, 0].set_title("PSNR: %0.4f " % the_psnr)
        axes[0, 0].imshow(model_output.cpu().view(256,256).detach().numpy())
        axes[0, 1].set_title("Our Gradient")
        axes[0, 1].imshow(img_grad.cpu().norm(dim=-1).view(256,256).detach().numpy())
        axes[0, 2].set_title("Our Laplacian")
        axes[0, 2].imshow(img_laplacian.cpu().view(256,256).detach().numpy())
        axes[1, 0].set_title("Ground Truth")
        axes[1, 0].imshow(gt['pixels'].cpu().view(256,256).detach().numpy())
        axes[1, 1].set_title("Ground Truth Gradient")
        axes[1, 1].imshow(gt['grads'].norm(dim=-1).cpu().view(256,256).detach().numpy())
        axes[1, 2].set_title("Ground Truth Laplacian")
        axes[1, 2].imshow(gt['laplace'].cpu().view(256,256).detach().numpy())
        plt.show()
        
    optim.zero_grad() #Sets the gradients of all optimized torch.Tensor to zero
    train_loss.backward() #apply gradients
    optim.step()  #make optimizer step

Output hidden; open in https://colab.research.google.com to view.

<a id='section_4'></a>
## *3.* Siren vs Relu

I will now compare Siren with a fully connected net using ReLU activation function

In [17]:
class ReluNet(nn.Module):
    def __init__(self, in_features, hidden_features, hidden_layers, out_features):
        super().__init__()
        
        self.net = []   #net

        #append the first fully connected layer
        self.net.append(nn.Linear(in_features, hidden_features))
        self.net.append(nn.ReLU())

        #append all the other fully connected layers
        for i in range(hidden_layers):
            self.net.append(nn.Linear(hidden_features, hidden_features))
            self.net.append(nn.ReLU())

        #append the final fully connected layer
        self.net.append(nn.Linear(hidden_features, out_features))

        
        
        #nn.Sequential(*self.net):
        #A sequential container. Modules will be added to it in the order they
        #are passed in the constructor.

        self.net = nn.Sequential(*self.net)
    
    def forward(self, coords):  #forward pass
        coords = coords.clone().detach().requires_grad_(True) # allows to take derivative w.r.t. input
        output = self.net(coords) #apply coords on net
        return output, coords        

    def forward_with_activations(self, coords, retain_grad=False):  #forward pass with activations
        '''Returns not only model output, but also intermediate activations.
        Only used for visualizing activations later!'''
        activations = OrderedDict()

        activation_count = 0
        x = coords.clone().detach().requires_grad_(True)  # allows to take derivative w.r.t. input
        activations['input'] = x
        for i, layer in enumerate(self.net):  #for all layers of the net
            if isinstance(layer, SineLayer):  #if it is a sine layer
                x, intermed = layer.forward_with_intermediate(x)
                
                #retain_grad():
                #Enables .grad attribute for non-leaf Tensors.

                if retain_grad:
                    x.retain_grad()
                    intermed.retain_grad()
                    
                activations['_'.join((str(layer.__class__), "%d" % activation_count))] = intermed
                activation_count += 1
            else: 
                x = layer(x)
                
                if retain_grad:
                    x.retain_grad()
                    
            activations['_'.join((str(layer.__class__), "%d" % activation_count))] = x
            activation_count += 1

        return activations

In [18]:
#free some memory
try:
    img_siren
except NameError:
    print("first time")
else:
    del img_siren

try:
    img_relu
except NameError:
    print("first time")
else:
    del img_relu

try:
    optim
except NameError:
    print("first time")
else:
    del optim

try:
    model_input
except NameError:
    print("first time")
else:
    del model_input

try:
    model_output
except NameError:
    print("first time")
else:
    del model_output

try:
    img_grad
except NameError:
    print("first time")
else:
    del img_grad

try:
    img_laplacian
except NameError:
    print("first time")
else:
    del img_laplacian

try:
    new_model_input
except NameError:
    print("first time")
else:
    del new_model_input

try:
    new_model_output
except NameError:
    print("first time")
else:
    del new_model_output

try:
    new_ground_truth
except NameError:
    print("first time")
else:
    del new_ground_truth

try:
    new_coords
except NameError:
    print("first time")
else:
    del new_coords

first time
first time
first time
first time
first time


In [19]:
cameraman = ImageFitting(256)

#instantiate data loader with a single batch (we have only one image)
dataloader = DataLoader(cameraman, batch_size=1, pin_memory=True, num_workers=0)

#instantiate siren net with number of input features = 2 ((x1,x2) pairs),
#number of output features = 1 (y value), hidden features, hidden layers
#and if to have the last layer as a linear layer or not
img_siren = Siren(in_features=2, out_features=1, hidden_features=256, 
                  hidden_layers=2, outermost_linear=True)

img_relu = ReluNet(in_features=2, out_features=1, hidden_features=256, 
                  hidden_layers=2)

img_siren.cuda(), img_relu.cuda()

relu_total_params = sum(p.numel() for p in img_relu.parameters() if p.requires_grad)
print("Relu net parameters: %d" % relu_total_params)
siren_total_params = sum(p.numel() for p in img_relu.parameters() if p.requires_grad)
print("Siren net parameters: %d" % siren_total_params)

Relu net parameters: 132609
Siren net parameters: 132609


### Siren

In [20]:
total_steps = 500 # Since the whole image is our dataset, this just means 500 gradient descent steps.
steps_til_summary = 10 #steps at which print image and summary

#define optimizer
optim = torch.optim.Adam(lr=1e-4, params=img_siren.parameters())

#model_input is the grid (of (x1,x2) pairs)
#ground_truth is the image (y values)
model_input, ground_truth = next(iter(dataloader))
model_input, ground_truth = model_input.cuda(), ground_truth.cuda()

for step in range(total_steps):

    #model_output is the output of the single forward pass, it is an output image
    #tensor of dimension (sidelength x sidelength, 1);
    #this represents the y^ (output) value for each pixel

    #coords is a copy of the model_input pairs
    model_output, coords = img_siren(model_input) #forward pass 

    #model_output and ground_truth (original image) have the same dimensions
    loss = ((model_output - ground_truth)**2).mean()  #calculate loss (MSE)
    the_psnr = psnr(ground_truth, model_output)
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        img_grad = gradient(model_output, coords) #calculate output image gradient
        img_laplacian = laplace(model_output, coords) #calculate output image laplacian

        fig, axes = plt.subplots(1,4, figsize=(18,6))
        axes[0].set_title("PSNR: %0.4f" % the_psnr)
        axes[0].imshow(model_output.cpu().view(256,256).detach().numpy())
        axes[1].set_title("Gradient")
        axes[1].imshow(img_grad.norm(dim=-1).cpu().view(256,256).detach().numpy())
        axes[2].set_title("Laplacian")
        axes[2].imshow(img_laplacian.cpu().view(256,256).detach().numpy())
        axes[3].set_title("Ground Truth")
        axes[3].imshow(ground_truth.cpu().view(256,256).detach().numpy())

        plt.show()

    optim.zero_grad() #Sets the gradients of all optimized torch.Tensor to zero
    loss.backward() ##apply gradients
    optim.step()  #make optimizer step

Output hidden; open in https://colab.research.google.com to view.

### ReLU

In [21]:
total_steps = 500 # Since the whole image is our dataset, this just means 500 gradient descent steps.
steps_til_summary = 10 #steps at which print image and summary

#define optimizer
optim = torch.optim.Adam(lr=1e-4, params=img_relu.parameters())

#model_input is the grid (of (x1,x2) pairs)
#ground_truth is the image (y values)
model_input, ground_truth = next(iter(dataloader))
model_input, ground_truth = model_input.cuda(), ground_truth.cuda()

for step in range(total_steps):

    #model_output is the output of the single forward pass, it is an output image
    #tensor of dimension (sidelength x sidelength, 1);
    #this represents the y^ (output) value for each pixel

    #coords is a copy of the model_input pairs
    model_output, coords = img_relu(model_input) #forward pass 

    #model_output and ground_truth (original image) have the same dimensions
    loss = ((model_output - ground_truth)**2).mean()  #calculate loss (MSE)
    
    if not step % steps_til_summary:
        print("Step %d, Total loss %0.6f" % (step, loss))
        img_grad = gradient(model_output, coords) #calculate output image gradient
        img_laplacian = laplace(model_output, coords) #calculate output image laplacian
        the_psnr = psnr(ground_truth, model_output)

        fig, axes = plt.subplots(1,4, figsize=(18,6))
        axes[0].set_title("PSNR: %0.4f" % the_psnr)
        axes[0].imshow(model_output.cpu().view(256,256).detach().numpy())
        axes[1].set_title("Gradient")
        axes[1].imshow(img_grad.norm(dim=-1).cpu().view(256,256).detach().numpy())
        axes[2].set_title("Laplacian")
        axes[2].imshow(img_laplacian.cpu().view(256,256).detach().numpy())
        axes[3].set_title("Ground Truth")
        axes[3].imshow(ground_truth.cpu().view(256,256).detach().numpy())

        plt.show()

    optim.zero_grad() #Sets the gradients of all optimized torch.Tensor to zero
    loss.backward() ##apply gradients
    optim.step()  #make optimizer step

Output hidden; open in https://colab.research.google.com to view.